In [1]:
import numpy as np

In [2]:
import plotly.io as pio
pio.renderers.default = "browser"

In [8]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import networkx as nx
import plotly.express as px






pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



# CSV einlesen (Passe den Pfad an)
df = pd.read_csv("Downloads/Battle DB 23 06 2024 - battle_events(2).csv")





In [10]:
# Namen in neue Spalte kopieren für Ansetzungsform > norm_name

df["norm_name #1"] = df["Name #1"]
df["norm_name #2"] = df["Name #2"]

In [12]:
# Ansetzungsform bereinigen, Groß-Kleinschreibung

df["norm_name #1"] = df["norm_name #1"].str.title()
df["norm_name #2"] = df["norm_name #2"].str.title()
df["hidden"] = df["hidden"].str.title()

In [14]:
# Ansetzungsform bereinigen, Klammern entfernen

df["norm_name #1"] = df["norm_name #1"].str.replace(r"\(.*\)", "", regex=True).str.strip()
df["norm_name #2"] = df["norm_name #2"].str.replace(r"\(.*\)", "", regex=True).str.strip()

In [16]:
# einzelne Spalten individuell anpassen

df.loc[1037, ["Name #1", "hidden"]] = ["Ssynic", "BMCL Titelmatch"]
df.loc[1255, ["Name #1", "hidden"]] = ["Besser", "BMCL meets DLTLLY Rap Battle"]
df.loc[688, ["hidden"]] = ["Merlin abwesend, Craze kickt seine Parts alleine"]
df.loc[198, ["Name #2", "hidden"]] = ["Shizu", "Deelah abwesend, Shizu springt ein"]
df.loc[440, ["norm_name #2", "Location", "hidden"]] = ["Jollyjay", "Kontext", "Der Sir abwesend, JollyJay springt ein"]
df.loc[966, ["Name #1", "Name #2", "hidden"]] = ["Ekhead & CPE", "Brian Damage & Karma", "Sport Rap-Battle, DLTLLY & SPOX: Borussia Dortmund vs. FC Bayern München"]

In [18]:
# Freestyle-Battles, Ankündigungen, Interviews, 2on2-Battles rausfiltern > df_battles

df_battles = df[~df["hidden"].str.contains('Interview|Ankündigung|Freestyle', na=False)]
df_battles = df_battles[~df_battles["Name #1"].str.contains(r'&|Freestyle|Ankündigung|PPV|\+', na=False)]
df_battles = df_battles[~df_battles["Name #2"].str.contains(r'&|Freestyle|Ankündigung|PPV|\+', na=False)]

In [22]:
# Aliase auflisten für norm_name-Spalten

aliases = {
    # Triple
    "Triplebeat": "Triple",
    "Triple Beat": "Triple",
    "TRIPLE": "Triple",  
    
    ".Tecey": "Tecey",
    
    "Alice": "Atropatrap",
    "Atzenkalle -": "Atzenkalle",

    "Bahamakarma": "Karma",
    "B-Dad": "Bdad",
    "Andres Kaliba": "Bilal",
    "Bong Teggy - Rematch": "Bong Teggy",
    "Bronko Banano Aka Street": "Bronko Banano",
    "Street": "Bronko Banano",
    
    "Cashus K.": "Cashus K",
    "Capco Jonnes": "Capco Jones",
    "Capcojones": "Capco Jones",
    "Cemo": "Cemo Dafuq",
    "Change": "Change19",
    "Cleptomatic": "Clep",
    "Clepto": "Clep",
    "Craze' Parts": "Craze",

    "D.E.Z.": "D.E.Z",
    "Der Fischer -": "Der Fischer",
    "Desperados": "Despo",

    "Ein Letztes Mal: Yarambo": "Yarambo",

    "Falk": "Falki",
    "Fatcap -": "Fatcap",
    "Fate One": "Fate",
    "Fate1": "Fate",
    "Four Seven": "Fourseven",
    "Fraxone": "Frax One",

    "Gier -": "Gier",

    "Hansen - Rematch": "Hansen",
    "Herrn Sören": "Herr Sören",

    "Jack Dragon - Rematch!": "Jack Dragon",
    "Jean Gough Frais": "Jean-Gough Frais",
    "Jizi": "Ji-Zi",
    "Juse Ju -": "Juse Ju",

    "Khacoby": "Karkobi",
    "King Lil&#39;C'": "King Lil'C",
    "Kol&#39;ja": "Kol'ja",

    "Lavas": "Lavaş",

    "Main Moe -": "Main Moe",
    "Malik N.": "Malik",
    "Mars": "Mars B.",
    "Mighty P.": "Mighty P",
    "Mighty Mo": "Meidi",

    "Nichtdeintyp": "Nicht Dein Typ",

    "O&#39;Spella": "O'Spella",
    
    "P-Zak -": "P-Zak",
    "Pueblo Escobar": "Pablo",
    "Papi Schlauch -": "Papi Schlauch",
    "Phillie": "Phillie.45",
    "Presto -": "Presto",
    "Pretty Shitty Jiggy": "PrettyShittyJiggy",
    "Prettyshittyjiggy": "Pretty Shitty Jiggy",

    "Quasi": "Quasi Infamous",

    "Ryko J": "Ryko-J",

    "Schlawinor": "Schlawin0r",
    "Schlawin0R": "Schlawin0r",
    "Snakeheadrhyme": "SnakeHeadRhyme",

    "Tableddn Timmy": "TableddnTimmy",
    "Tableddntimmy": "TableddnTimmy",
    "Tierstar -": "Tierstar",
    "Tightammic -": "Tightammic",

    "Mr. Whyte": "Yaamann",
    "Yamann": "Yaamann",
    "Mr. White": "Yaamann",
    "Mr White": "Yaamann"
    
    
    
}

df_battles["norm_name #1"] = df_battles["norm_name #1"].replace(aliases)
df_battles["norm_name #2"] = df_battles["norm_name #2"].replace(aliases)

In [24]:
#df_battles_clean.loc[440, ["Name #2", "Location", "hidden"]] = ["Jollyjay", "Kontext", "Ersatz für 'Der Sir'"]
#df_battles_clean.loc[235, ["Name #2", "Event", "Location", "Stadt", "hidden"]] = ["Hiding John", "NRWeekend", "Spektakulum", "Düsseldorf", "One Rounder"]

df_battles.loc[1037]

Name #1                                              Ssynic
Name #2                                           MIGHTY MO
Event                                                   NaN
Location                                                NaN
Stadt                                                   NaN
Type                                              Accapella
Year                                                   2017
Channel                                                 RAM
Uploaded                                         2017-06-30
URL             https://www.youtube.com/watch?v=FbeBGeRf4F0
Views                                                694283
ID                                              FbeBGeRf4F0
hidden                                      BMCL Titelmatch
Processed                     2024-08-03T02:43:42.957-04:00
norm_name #1                        Bmcl Titelmatch: Ssynic
norm_name #2                                          Meidi
Name: 1037, dtype: object

In [26]:

df_battles[df_battles["norm_name #2"].str.contains('Jolly', na=False)]


,Name #1,Name #2,Event,Location,Stadt,Type,Year,Channel,Uploaded,URL,Views,ID,hidden,Processed,norm_name #1,norm_name #2
364,Yarambo,JollyJay,NaN,NaN,NaN,Accapella,2023,DLTLLY,2023-10-29,https://www.youtube.com/watch?v=u6NT4ynzzXY,112586,u6NT4ynzzXY,B-Day #10,2024-08-03T02:43:42.877-04:00,Yarambo,Jollyjay
440,S-Staticx,Der Sir (JollyJay),Proving Grounds,Kontext,Wiesbaden,Accapella,2023,DLTLLY,2023-03-31,https://www.youtube.com/watch?v=Od-__geija4,9544,Od-__geija4,"Der Sir abwesend, JollyJay springt ein",2024-08-03T02:43:42.887-04:00,S-Staticx,Jollyjay
472,Mars B.,JollyJay,NaN,NaN,Kassel,Accapella,2023,DLTLLY,2023-01-08,https://www.youtube.com/watch?v=gGQdvDUSOpo,115408,gGQdvDUSOpo,Kassel,2024-08-03T02:43:42.891-04:00,Mars B.,Jollyjay
749,Dizaster,JollyJay,NaN,NaN,NaN,Accapella,2019,DLTLLY,2019-10-26,https://www.youtube.com/watch?v=1ap7X5HV54k,103658,1ap7X5HV54k,International Battle B.Day#6,2024-08-03T02:43:42.924-04:00,Dizaster,Jollyjay
980,Unanymous,JollyJay,NaN,NaN,NaN,Accapella,2017,DLTLLY,2017-10-29,https://www.youtube.com/watch?v=E4_2vzM1SCE,28961,E4_2vzM1SCE,B.Day#4,2024-08-03T02:43:42.951-04:00,Unanymous,Jollyjay
1310,INNUENDO,JOLLYJAY,NaN,NaN,NaN,Accapella,2014,DON'T FLOP,2014-09-10,https://www.youtube.com/watch?v=IHAEQ3v_qXg,29094,IHAEQ3v_qXg,Could Not Identify Event,2024-08-03T02:43:42.987-04:00,Innuendo,Jollyjay


In [28]:
# Battlerapper mit nur 2 Battles (beide Contender) rausfiltern > df_battles_clean

# Alle Namen aus beiden Spalten zählen
all_names = pd.concat([df_battles["norm_name #1"], df_battles["norm_name #2"]])
counts = all_names.value_counts()

# Bedingung: mindestens einer der beiden Namen kommt mehr als einmal vor
mask = df_battles.apply(lambda row: counts[row["norm_name #1"]] > 3 or counts[row["norm_name #2"]] > 3, axis=1)

# Gefiltertes DataFrame
df_battles_clean = df_battles[mask]


df_battles_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1101 entries, 2 to 1410
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name #1       1101 non-null   object
 1   Name #2       1101 non-null   object
 2   Event         160 non-null    object
 3   Location      110 non-null    object
 4   Stadt         362 non-null    object
 5   Type          1101 non-null   object
 6   Year          1101 non-null   int64 
 7   Channel       1101 non-null   object
 8   Uploaded      1101 non-null   object
 9   URL           1101 non-null   object
 10  Views         1101 non-null   int64 
 11  ID            1101 non-null   object
 12  hidden        1101 non-null   object
 13  Processed     998 non-null    object
 14  norm_name #1  1101 non-null   object
 15  norm_name #2  1101 non-null   object
dtypes: int64(2), object(14)
memory usage: 146.2+ KB


In [30]:
all_names = pd.concat([df_battles["norm_name #1"], df_battles["norm_name #2"]])
unique_names = sorted(all_names.unique())
unique_names

['4Tune',
 'A-Ron',
 'Abrupt',
 'Adam',
 'Adhs-Andy',
 'Akanoo',
 'Akzent',
 'Aladin',
 'Ali Affront',
 'Ali Smart',
 'Alko',
 'Amah',
 'Ams',
 'Andres',
 'Andy Amnesia',
 'Ano-Nym',
 'Apriori',
 'Aris',
 'Ars',
 'Atropatrap',
 'Atzenkalle',
 'Audienzz',
 'Aura',
 'Aytee',
 'Balavababa',
 'Banal',
 'Bandan',
 'Baretta',
 'Barracuda',
 'Battleboi Basti',
 'Bauda',
 'Bault',
 'Bayze',
 'Bdad',
 'Beastboy',
 'Belial',
 'Besser',
 'Bilal',
 'Black Snowman',
 'Bladesa',
 'Bmcl Meets Dltlly Rap Battle: Besser',
 'Bmcl Titelmatch: Ssynic',
 'Bong Teggy',
 'Brahmc',
 'Breeze',
 'Brian Damage',
 'Bronko Banano',
 'Bruda Tuck',
 'Bruf',
 'Bubba T',
 'Buddi',
 'Bungee',
 'Bunnee',
 'Burst',
 'Butchbruda',
 'Buy Some',
 'Bx',
 'Béla Garbage',
 'Calli',
 'Capco Jones',
 'Cartoon',
 'Cashisclay',
 'Cashus K',
 'Celune',
 'Cemo Dafuq',
 'Chaarlz',
 'Change19',
 'Chinooka',
 'Ciro',
 'Cizero',
 'Clep',
 'Clockwork',
 'Cloudsurfer',
 'Core One',
 'Corner',
 'Cpe',
 'Crash Bundy',
 'Craze',
 'Crazyo',
 

In [32]:


# Beispiel-Battles (Name1, Name2, Jahr, Ort, Gewinner)
#battles = [
 #   ("Shortworth", "Craze", 2019, "Wiesbaden", "Kontext", "Craze"),
  #  ("Shortworth", "Tao", 2018, "Frankfurt", "Zoom", "Tao"),
   # ("Shortworth", "Ryko-J", 2025, "Freiburg", "Location", "Shortworth"),
    #("Craze", "Neilz", 2024, "Berlin", "Festsaal Kreuzberg", "Neilz"),
#    ("Ryko-J", "Lucky Lily", 2025, "NRW", "Spektakulum", "unjudged"),
 #   ("Neilz", "Tobi High", 2023, "NRW", "Spektakulum", "unjudged"),
  #  ("Tobi High", "Ryko-J", 2019, "Bochum", "Die Trompete", "unjudged")
#]




battles = []
for _, row in df_battles_clean.iterrows():
     battles.append((row['norm_name #1'], row['norm_name #2'], int(row['Year']), row['Name #1'], row['Name #2'], row['Stadt'], row.get('Location', '')))

champs = ["Nedal Nib", "Yarambo", "Mikesh", "Kato"]

#node_symbols = []
#node_colors = []
#for node in G.nodes():
#    if node in champs:
#        node_symbols.append("star")
#        node_colors.append("gold")
#    else:
#        node_symbols.append("circle")
#        node_colors.append("grey")

# Graph aufbauen
G = nx.Graph()
for nn1, nn2, jahr, n1, n2, ort, location in battles:
    battle_info = f"{n1} vs. {n2}, {jahr} @ {ort}, {location}"
    G.add_node(nn1)    # benennung der Punkte
    G.add_node(nn2)
    if G.has_edge(nn1, nn2):
        G[nn1][nn2]['info'].append(battle_info)
        G[nn1][nn2]['years'].append(jahr)
    else:
        G.add_edge(nn1, nn2, info=[battle_info], years=[jahr])

pos = nx.spring_layout(G, seed=42)

colorscale = px.colors.sequential.Turbo  # sehr bunte, breite Farbpalette

def interpolate_color(norm):
    # norm 0..1
    idx = int(norm * (len(colorscale)-1))
    return colorscale[idx]

# Jahreswerte für Normierung
years_all = [jahr for _, _, jahr, _, _, _, _ in battles]
min_year, max_year = min(years_all), max(years_all)

# Kanten erstellen mit Jahresfarbe
edge_traces = []
for u, v, data in G.edges(data=True):
    x0, y0 = pos[u]
    x1, y1 = pos[v]

    # Durchschnittsjahr für Farbskala
    avg_year = sum(data['years']) / len(data['years'])
    norm = (avg_year - min_year) / (max_year - min_year) if max_year != min_year else 0
    color = interpolate_color(norm)

    # alle Battle-Infos der Kante in Tooltip packen
    info_text = "<br>".join(data['info'])

      # Punkte entlang der Linie erzeugen
    num_points = 50
    x_vals = np.linspace(x0, x1, num_points)
    y_vals = np.linspace(y0, y1, num_points)
    texts = [info_text] * num_points

    edge_traces.append(go.Scatter(
        x=x_vals,
        y=y_vals,
        line=dict(width=2, color=color),
        mode="lines+markers",   # Linie + Punkte (unsichtbar)
        marker=dict(size=0, opacity=0),   # Marker unsichtbar
        hoverinfo="text",
        text=texts
    ))

# Knoten vorbereiten
# normale Punkte
normal_nodes = [n for n in G.nodes() if n not in champs]
normal_x = [pos[n][0] for n in normal_nodes]
normal_y = [pos[n][1] for n in normal_nodes]
normal_text = [n for n in normal_nodes]

trace_normal = go.Scatter(
    x=normal_x, y=normal_y,
    mode="markers+text",
    text=normal_text,
    textposition="top center",
    hoverinfo="text",
    marker=dict(
        size=7,
        color="grey",
        symbol="circle",
        opacity=0.6,
        line=dict(width=1, color="black")  # Randfarbe + Stärke
    )
)

# hervorgehobene Sterne
star_nodes = champs
star_x = [pos[n][0] for n in star_nodes]
star_y = [pos[n][1] for n in star_nodes]
star_text = [n for n in star_nodes]

trace_stars = go.Scatter(
    x=star_x, y=star_y,
    mode="markers+text",
    text=star_text,
    textposition="top center",
    hoverinfo="text",
    marker=dict(
        size=10,       # größer als normale Punkte
        color="gold",
        symbol="star",
        opacity=0.8,
        line=dict(width=1, color="black")  # Randfarbe + Stärke
    )
)



# Plot erstellen
fig = go.Figure(data=edge_traces,
                layout=go.Layout(
                    title="Battle-Netzwerk mit Jahresfarben",
                    title_x=0.5,
                    showlegend=False,
                    hovermode="closest",
                    margin=dict(b=20,l=5,r=5,t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                ))

fig.add_traces([trace_normal, trace_stars])

fig.show()






In [614]:
df_battles['Year'].array

<NumpyExtensionArray>
[2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025, 2025,
 ...
 2010, 2011, 2011, 2011, 2011, 2011, 2010, 2012, 2012, 2010]
Length: 1248, dtype: int64

In [219]:

# 1. Zeilen finden, die Klammern enthalten
mask_parens = df_battles_clean["Name #1"].str.contains(r"\(.*\)", na=False) | df_battles_clean["Name #2"].str.contains(r"\(.*\)", na=False)

# 2. Diese Zeilen in separatem DataFrame speichern
df_with_parens = df_battles_clean[mask_parens]


df_with_parens

,Name #1,Name #2,Event,Location,Stadt,Type,Year,Channel,Uploaded,URL,Views,ID,hidden,Processed,norm_name #1,norm_name #2
249,PHILLIE.45,NASTIC (Freestyle als MAT),NaN,NaN,NaN,On Beat,2024,FOB,2024-05-24,https://www.youtube.com/watch?v=RzVabkX8At8,4636,RzVabkX8At8,On Beat Battle,2024-08-03T02:43:42.862-04:00,Phillie.45,Nastic
573,OREGANO,BUNNEE (NEWCOMER BATTLE),NaN,NaN,NaN,Accapella,2022,FOB,2022-04-22,https://www.youtube.com/watch?v=IpAK7JAG9aY,5363,IpAK7JAG9aY,Acapella Battle,2024-08-03T02:43:42.902-04:00,Oregano,Bunnee
575,NICK SHIT,BARETTA (NEWCOMER BATTLE),NaN,NaN,NaN,Accapella,2022,FOB,2022-04-15,https://www.youtube.com/watch?v=UHTFPhhnook,1613,UHTFPhhnook,Acapella Battle,2024-08-03T02:43:42.903-04:00,Nick Shit,Baretta
577,CRAZYO,M.F.L. (NEWCOMER BATTLE),NaN,NaN,NaN,Accapella,2022,FOB,2022-04-08,https://www.youtube.com/watch?v=dwxhHY6EU6Y,1922,dwxhHY6EU6Y,Acapella Battle,2024-08-03T02:43:42.903-04:00,Crazyo,M.F.L.
579,PESC,LEOQUENT (NEWCOMER BATTLE),NaN,NaN,NaN,Accapella,2022,FOB,2022-03-25,https://www.youtube.com/watch?v=1FsbrboRc0c,1635,1FsbrboRc0c,Acapella Battle,2024-08-03T02:43:42.903-04:00,Pesc,Leoquent
580,FROZEN,EZRA (NEWCOMER BATTLE),NaN,NaN,NaN,Accapella,2022,FOB,2022-03-24,https://www.youtube.com/watch?v=6LKmFzyGKjs,1699,6LKmFzyGKjs,Acapella Battle,2024-08-03T02:43:42.903-04:00,Frozen,Ezra
629,Joseph Steinschleuder (aka Merlin),Tobi High,NaN,NaN,Berlin,Accapella,2021,DLTLLY,2021-10-15,https://www.youtube.com/watch?v=ErS3GzWV5EI,39140,ErS3GzWV5EI,B-Day #8 Berlin,2024-08-03T02:43:42.909-04:00,Joseph Steinschleuder,Tobi High
637,Kato,Indoor Stan (OnBeat),NaN,NaN,NaN,On Beat,2021,DLTLLY,2021-08-01,https://www.youtube.com/watch?v=X6kOp_8oxBU,29369,X6kOp_8oxBU,Summer Vibes 2021,2024-08-03T02:43:42.910-04:00,Kato,Indoor Stan
656,DOC,Gerro (aka RenetixX),NaN,NaN,NaN,On Beat,2021,DLTLLY,2021-02-07,https://www.youtube.com/watch?v=FYm1CMj1TJQ,11071,FYm1CMj1TJQ,Video Battle,2024-08-03T02:43:42.912-04:00,Doc,Gerro
948,ShaoLum,Joe-L (Sport-Bars),NaN,NaN,NaN,Accapella,2018,SPOX,2018-01-02,https://www.youtube.com/watch?v=y7s-nFdjpQg,35335,y7s-nFdjpQg,Could Not Identify Event,2024-08-03T02:43:42.948-04:00,Shaolum,Joe-L


In [263]:
# Liste der rausgeschmissenen nur-2-Battles
df_one_battle = df_battles[~mask]
df_one_battle

,Name #1,Name #2,Event,Location,Stadt,Type,Year,Channel,Uploaded,URL,Views,ID,hidden,Processed,norm_name #1,norm_name #2
1,TOBO,JAVAS,NaN,NaN,NaN,Accapella,2025,FOB,2025-09-26,https://www.youtube.com/watch?v=aS2dhmR_Q9U,2110,aS2dhmR_Q9U,Rap Battle,NaN,Tobo,Javas
2,Lara Soft,Freezy,NaN,NaN,NaN,Accapella,2025,DLTLLY,2025-09-23,https://www.youtube.com/watch?v=_luCDi3t-3w,11308,_luCDi3t-3w,Taped!,NaN,Lara Soft,Freezy
8,RIMA,MANEK,NaN,NaN,NaN,Accapella,2025,FOB,2025-09-07,https://www.youtube.com/watch?v=Wn_Tm08TPlo,3561,Wn_Tm08TPlo,Rap Battle,NaN,Rima,Manek
11,BAULT,SILVER,NaN,NaN,NaN,Accapella,2025,FOB,2025-08-31,https://www.youtube.com/watch?v=SzXSJYcHl80,3525,SzXSJYcHl80,Rap Battle,NaN,Bault,Silver
13,JIZI,KEULE257,NaN,NaN,NaN,Accapella,2025,FOB,2025-08-29,https://www.youtube.com/watch?v=PlRe0-7MdtE,28084,PlRe0-7MdtE,Rap Battle,NaN,Jizi,Keule257
15,L-SUPREMA,LYT,NaN,NaN,NaN,Accapella,2025,FOB,2025-08-24,https://www.youtube.com/watch?v=f4lsqfYo37M,3596,f4lsqfYo37M,Rap Battle,NaN,L-Suprema,Lyt
18,Frax One,Danny Deadweight,NaN,NaN,NaN,Accapella,2025,DLTLLY,2025-08-21,https://www.youtube.com/watch?v=8DkFnov9Fow,5463,8DkFnov9Fow,Proving Grounds,NaN,Frax One,Danny Deadweight
21,TRIPLE,RALLE,NaN,NaN,NaN,Accapella,2025,FOB,2025-08-17,https://www.youtube.com/watch?v=VlMReN-CMX0,3000,VlMReN-CMX0,Rap Battle,NaN,Triple,Ralle
24,TRACY,MZUDEMK,NaN,NaN,NaN,Accapella,2025,FOB,2025-08-10,https://www.youtube.com/watch?v=QyQ2poDBDYM,12061,QyQ2poDBDYM,Game Changer Viertelfinale,NaN,Tracy,Mzudemk
25,AMS,JÄL,NaN,NaN,NaN,Accapella,2025,FOB,2025-08-09,https://www.youtube.com/watch?v=eZLTR2OLG7M,9168,eZLTR2OLG7M,Game Changer Viertelfinale,NaN,Ams,Jäl
